In [25]:
import os
import pandas as pd
from tqdm import tqdm
from random import sample  # 随机采样

In [5]:
# 读取数据集
df_train_click = pd.read_csv('../dataset/train_click_log.csv')
df_test_click = pd.read_csv('../dataset/testA_click_log.csv')
print(f'训练集样本数：{len(df_train_click)}   测试集样本数：{len(df_test_click)}')

训练集样本数：1112623   测试集样本数：518010


In [17]:
# 随机采样出一部分数据
train_users = df_train_click['user_id'].values.tolist()
val_users = (sample(train_users, 50000))
len(train_users), len(val_users)

(1112623, 50000)

In [21]:
# 训练集用户 抽出行为数据最后一条作为线下验证集
click_list = valid_query_list = []
groups = df_train_click.groupby(['user_id'])
for user_id, g in tqdm(groups):
    if user_id in val_users:
        valid_query_list.append(g.tail(1)[['user_id', 'click_article_id']])
        click_list.append(g.head(g.shape[0] - 1))
    else:
        click_list.append(g)

100%|██████████| 200000/200000 [05:43<00:00, 582.75it/s]


In [22]:
df_train_click = pd.concat(click_list, sort=False)
df_valid_query = pd.concat(valid_query_list, sort=False)

In [23]:
# 处理测试集
test_users = df_test_click['user_id'].unique()
test_query_list = []
for user in tqdm(test_users):
    test_query_list.append([user, -1])

df_test_query = pd.DataFrame(test_query_list, columns=['user_id', 'click_article_id'])

100%|██████████| 50000/50000 [00:00<00:00, 1651808.04it/s]


In [24]:
df_query = pd.concat([df_valid_query, df_test_query], sort=False).reset_index(drop=True)
df_click = pd.concat([df_train_click, df_test_click], sort=False).reset_index(drop=True)
df_click = df_click.sort_values(['user_id', 'click_timestamp']).reset_index(drop=True)

In [26]:
# 保存数据
os.makedirs('../data/offline', exist_ok=True)
df_click.to_pickle('../data/offline/click.pkl')
df_query.to_pickle('../data/offline/query.pkl')